# Top & Bottom 10 Neighbourhoods Bar Chart

Horizontal bar chart showing the highest and lowest scoring neighbourhoods by livability.

## 1. Data Reference

### Source Tables

| Table | Grain | Key Columns |
|-------|-------|-------------|
| `mart_neighbourhood_overview` | neighbourhood × year | neighbourhood_name, livability_score |

### SQL Query

In [1]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load .env from project root
load_dotenv("../../.env")

engine = create_engine(os.environ["DATABASE_URL"])

query = """
SELECT
    neighbourhood_name,
    livability_score,
    safety_score,
    affordability_score,
    amenity_score
FROM mart_toronto.mart_neighbourhood_overview
WHERE year = (SELECT MAX(year) FROM mart_toronto.mart_neighbourhood_overview)
  AND livability_score IS NOT NULL
ORDER BY livability_score DESC
"""

df = pd.read_sql(query, engine)
print(f"Loaded {len(df)} neighbourhoods with scores")

Loaded 158 neighbourhoods with scores


### Transformation Steps

1. Sort by livability_score descending
2. Take top 10 and bottom 10
3. Pass to ranking bar figure factory

In [2]:
# The figure factory handles top/bottom selection internally
# Just prepare as list of dicts
data = df.to_dict("records")

### Sample Output

In [3]:
print("Top 5:")
display(df.head(5))
print("\nBottom 5:")
display(df.tail(5))

Top 5:


,neighbourhood_name,livability_score,safety_score,affordability_score,amenity_score
0,Lambton Baby Point,90.0,100.0,100.0,50.0
1,Henry Farm,89.7,99.4,100.0,50.0
2,Centennial Scarborough,89.5,98.7,100.0,50.0
3,Guildwood,89.2,98.1,100.0,50.0
4,Steeles,89.0,97.5,100.0,50.0



Bottom 5:


,neighbourhood_name,livability_score,safety_score,affordability_score,amenity_score
153,University,51.0,2.5,100.0,50.0
154,Downtown Yonge East,50.8,1.9,100.0,50.0
155,Kensington-Chinatown,50.5,1.3,100.0,50.0
156,Yonge-Bay Corridor,50.3,0.6,100.0,50.0
157,Mimico-Queensway,50.0,0.0,100.0,50.0


## 2. Data Visualization

### Figure Factory

Uses `create_ranking_bar` from `portfolio_app.figures.toronto.bar_charts`.

**Key Parameters:**
- `data`: List of dicts with all neighbourhoods
- `name_column`: 'neighbourhood_name'
- `value_column`: 'livability_score'
- `top_n`: 10 (green bars)
- `bottom_n`: 10 (red bars)

In [4]:
import sys

sys.path.insert(0, "../..")

from portfolio_app.figures.toronto.bar_charts import create_ranking_bar

fig = create_ranking_bar(
    data=data,
    name_column="neighbourhood_name",
    value_column="livability_score",
    title="Top & Bottom 10 Neighbourhoods by Livability",
    top_n=10,
    bottom_n=10,
    color_top="#4CAF50",  # Green for top performers
    color_bottom="#F44336",  # Red for bottom performers
    value_format=".1f",
)

fig.show()

### Interpretation

- **Green bars**: Highest livability scores (best combination of safety, affordability, and amenities)
- **Red bars**: Lowest livability scores (areas that may need targeted investment)

The ranking bar chart provides quick context for which neighbourhoods stand out at either extreme.